## <div style="color: #E98E29">**1. Ler PDF completo** </div>

 - <s style="color: #92BB2D">CPFL</s>
 - <s style="color: #92BB2D">COPEL</s>
 - <s style="color: #92BB2D">CEMIG</s>
 - <s style="color: #92BB2D">EDP</s>
 - ELEKTRO
 - ENEL

---

## <div style="color: #E98E29">**2. Criar a base**</div>
- <s style="color: #92BB2D">Coletar Meses e ano(s)</s>
- <s style="color: #92BB2D">Coletar Consumos</s>
-  <s style="color: #92BB2D">Coletar Demandas</s>
- <s style="color: #92BB2D">Criar DataFrame</s>
- Adicionar a base de dados
<br>
<br>
---

## <s style="color: #92BB2D">**3. Não adicionar datas repetidas**</s>

<br>

## LER OS PDFS

In [1]:
import sys

classes = r"C:\Users\mateus.souza\Desktop\Projetos_n\leitor_pdf\coleta\scripts\classes"
funcs = r"C:\Users\mateus.souza\Desktop\Projetos_n\leitor_pdf\coleta\scripts\funcoes"
sys.path.append(classes)
sys.path.append(funcs)

In [2]:
import os
import contextlib
import regex as re
import pandas as pd
from funcoes.f_database import *
from typing import Any, List, Tuple
from Fatura import Fatura

join_path = lambda y, x: os.path.join(y, x)
filename = lambda path, filetype: path.split("\\")[-1].split(filetype)[0]

PATH = r"Q:\APOIO ATENDIMENTO\Faturas EcoExp\Setembro"
FOLDERS = os.listdir(PATH)
PATHS = [{filename(x, ".pdf"): join_path(PATH, x)} for x in FOLDERS]

# SCRIPTS

## ADICIONAR IDS

In [ ]:
# ADICIONA NOVOS CLIENTES
df = []
ucs_df = df[["nome"]].drop_duplicates().reset_index(drop=True)
db_path = r"C:\Users\mateus.souza\Desktop\Projetos_n\leitor_pdf\banco_de_dados\distribuidoras.db"
ucs_db = get_clientes_existentes(db_path)
ucs_nova = adiciona_novo_id_clientes(mantem_novos_clientes(ucs_df, ucs_db))
if not (ucs_nova.empty):
    print(f"Clientes a adicionar: {', '.join(ucs_nova.nome.values.tolist())}")
    if salva_novos_clientes(ucs_nova, db_path):
        print("Insercao de Novos Clientes ok")
        ucs_db = get_clientes_existentes(db_path)
    else:
        print("Erro ao salvar valores no banco")
else:
    print("Nenhuma UC nova")

consumos_pdf = criar_id_consumos(df_id=ucs_db, df_consumo=df.drop_duplicates(subset=["nome", "datas"]))
consumos_db = get_consumos_existentes(db_path)
novos_consumos = mantem_novos_consumos(
    novos_consumos=consumos_pdf, consumos_existentes=consumos_db
)
if not(novos_consumos.empty):
    status = salva_novos_consumos(novos_consumos=novos_consumos, db_path=db_path)

## DATAFRAME FINAL

In [4]:
from Identificador import Identificador
from PyPDF2.errors import PdfReadError
DISTRIBUIDORAS = {}
for info in PATHS:
    with contextlib.suppress(IndexError,TypeError, PdfReadError):
        path = info.get(*info.keys())
        DISTRIBUIDORA = Identificador(infos=info).main()(path)
        name = DISTRIBUIDORA.distribuidora
        if name not in DISTRIBUIDORAS.keys():
            DISTRIBUIDORAS[name] = []
        if name != None:
            if name in ["CPFL", "EDP", "COPEL", "CEMIG"]:
                DISTRIBUIDORA.main()
            DISTRIBUIDORAS[name].append(DISTRIBUIDORA)


# NOVAS CLASSES - DESENVOLVIMENTO

In [ ]:
from EDP import EDP
from CPFL import CPFL
from COPEL import COPEL
from CEMIG import CEMIG
CONSUMOS = []

for cemig in DISTRIBUIDORAS['CEMIG']:
    caminho = cemig._caminho
    cmg = CEMIG(caminho)
    cmg.main()
    CONSUMOS.append(cmg.consumo)

for cpfl in DISTRIBUIDORAS['CPFL']:
    caminho = cpfl._caminho
    cpfl = CPFL(caminho)
    cpfl.main()
    CONSUMOS.append(cpfl.consumo)

for edp in DISTRIBUIDORAS['EDP']:
    caminho = edp._caminho
    edp = EDP(caminho)
    edp.main()
    CONSUMOS.append(edp.consumo)

for copel in DISTRIBUIDORAS['COPEL']:
    caminho = copel._caminho
    copel = COPEL(caminho)
    copel.main()
    CONSUMOS.append(copel.consumo)

In [12]:
CONSUMOS = []
for key in DISTRIBUIDORAS:
    if key in ["CPFL", "EDP", "COPEL", "CEMIG"]:
        CONSUMOS.extend(dis.demanda for dis in DISTRIBUIDORAS[key])

In [16]:
pd.concat(CONSUMOS, ignore_index=True)

,datas,demanda_fora_de_ponta,demanda_ponta,data
0,2023-09-01,369.6,NaN,NaT
1,2023-08-01,344.4,NaN,NaT
2,2023-07-01,364.0,NaN,NaT
3,2023-06-01,376.3,NaN,NaT
4,2023-05-01,375.8,NaN,NaT
...,...,...,...,...
2074,2023-01-01,241,239,NaT
2075,2022-12-01,318,286,NaT
2076,2022-11-01,241,286,NaT
2077,2022-10-01,245,221,NaT


In [ ]:
idx = cpfl._texto_posicao("consumo fora de ponta")

string = cpfl.ultima_pagina[idx:][:re.search("\n\w.*dias", cpfl.ultima_pagina[idx:]).span()[0]]
datas_lista = cpfl.datas_texto(re.sub("ll.*", "", string).split("\n")[1:])
datas_series = pd.Series(datas_lista, name="data")
cpfl._data = datas_series.apply(cpfl.transforma_data)

In [ ]:
cpfl.data

In [ ]:
erro = cpfl._caminho 